In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/02 20:14:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [21]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [22]:
df_green.registerTempTable("green_table")

/opt/homebrew/Cellar/apache-spark/3.5.0/libexec/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [23]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', lpep_pickup_datetime) AS revenue_hour, 
    PULocationID AS revenue_zone,

    -- Revenue calculation 
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green_table
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    revenue_hour, revenue_zone
ORDER BY
    revenue_hour, revenue_zone
""")

In [24]:
df_green_revenue.repartition(24)

DataFrame[revenue_hour: timestamp, revenue_zone: int, amount: double, number_records: bigint]

In [25]:
df_green_revenue \
    .repartition(24) \
    .write.parquet('data/report/revenue/green/', mode='overwrite')

24/03/02 20:20:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [26]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [27]:
df_yellow.registerTempTable("yellow_table")

In [29]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', tpep_pickup_datetime) AS revenue_hour, 
    PULocationID AS revenue_zone,

    -- Revenue calculation 
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow_table
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    revenue_hour, revenue_zone
ORDER BY
    revenue_hour, revenue_zone
""")

In [30]:
df_yellow_revenue.repartition(24)

DataFrame[revenue_hour: timestamp, revenue_zone: int, amount: double, number_records: bigint]

In [31]:
df_yellow_revenue \
    .repartition(24) \
    .write.parquet('data/report/revenue/yellow/', mode='overwrite')

24/03/02 20:21:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 20:21:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/02 20:21:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/02 20:21:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/02 20:21:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 20:21:04 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 20:21:04 WARN MemoryManager: Total allocation exceeds 95.00%

In [32]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records') 

In [33]:
df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records') 

In [37]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['revenue_hour', 'revenue_zone'], how='outer')

In [38]:
df_join.show()

+-------------------+------------+------------------+--------------------+-----------------+---------------------+
|       revenue_hour|revenue_zone|      green_amount|green_number_records|    yellow_amount|yellow_number_records|
+-------------------+------------+------------------+--------------------+-----------------+---------------------+
|2020-01-01 00:00:00|           3|              NULL|                NULL|             25.0|                    1|
|2020-01-01 00:00:00|          40|            168.98|                   8|            89.97|                    5|
|2020-01-01 00:00:00|          45|              NULL|                NULL|732.4800000000002|                   42|
|2020-01-01 00:00:00|          47|              13.3|                   1|              8.3|                    1|
|2020-01-01 00:00:00|          51|              17.8|                   2|             31.0|                    1|
|2020-01-01 00:00:00|          68|              NULL|                NULL|7825.0

In [39]:
df_join.write.parquet('data/report/revenue/total/', mode='overwrite')

24/03/02 20:24:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 20:24:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/02 20:24:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/02 20:24:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/02 20:24:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [40]:
df_join = spark.read.parquet('data/report/revenue/total/')

In [42]:
df_join.show()

+-------------------+------------+------------------+--------------------+------------------+---------------------+
|       revenue_hour|revenue_zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+------------+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|          22|              15.8|                   1|              NULL|                 NULL|
|2020-01-01 00:00:00|          33|317.27000000000004|                  11|            255.56|                    8|
|2020-01-01 00:00:00|          35|            129.96|                   5|              NULL|                 NULL|
|2020-01-01 00:00:00|          52| 83.33000000000001|                   4|              49.8|                    2|
|2020-01-01 00:00:00|          55|            129.29|                   4|              NULL|                 NULL|
|2020-01-01 00:00:00|          56|             99.69|                   

In [43]:
df_zones = spark.read.parquet('zones')

In [45]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [46]:
df_join_zone = df_join.join(df_zones, df_join.revenue_zone == df_zones.LocationID)

In [51]:
df_join_zone.drop('LocationID').write.parquet('tmp/revenue-zones', mode='overwrite')

24/03/02 20:39:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/03/02 20:39:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/02 20:39:23 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
24/03/02 20:39:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
24/03/02 20:39:24 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
